**Zadanie 2. (7p)** Zadanie to jest rozwinięciem zadania poprzedniego. Wyszukiwarka, którą piszesz
w tym punkcie powinna:

1. obsługiwać zapytania zwykłe i frazowe,
2. rozpoznawać typ zapytania po obecności cudzysłowów,
3. obsługiwać zapytania frazowe w dowolny sposób, niekoniecznie indeksem pozycyjnym,
4. tworzyć ranking dla zapytań zwykłych, który uwzględnia cechy z poprzedniej listy zadań,
5. premiując dodatkowo dokumenty zawierające frazy z zapytania.

Oznacza to w szczególności, że w przypadku zapytania zwykłego, będącego frazą, dokumenty zawierające tę frazę powinny być wysoko na liście odpowiedzi. Należy również inteligentnie obsługiwać
takie pytania jak:

> kodeks karny kara za morderstwo

i premiować dokumenty zawierające np. takie zdanie

> w ostatniej nowelizacji kodeksu karnego wprowadzono podwyższoną karę za morderstwo,

a nie premiować takiego

> Karni żołnierze mają kodeks, którego przekroczenie powoduje karę, a za jakiś czas nawet
morderstwo .

In [62]:
import csv
import logging
from collections import defaultdict
from bisect import bisect
from itertools import chain

In [2]:
BASE_FORMS_FILE_PATH = 'data/polimorfologik-2.1.txt'
WIKI_ARTICLES_FILE_PATH = 'data/fp_wiki.txt'

In [3]:
BASE_FORMS = {}

with open(BASE_FORMS_FILE_PATH) as f:
    for base_form, word, *_ in csv.reader(f, delimiter=';'):
        BASE_FORMS[word.lower()] = base_form

In [4]:
def base(word):
    return BASE_FORMS.get(word.lower())

In [5]:
POSITION_BY_ID = []
WIKI_ARTICLES = []
POSITIONAL_INDEX = defaultdict(set)

with open(WIKI_ARTICLES_FILE_PATH) as f:
    lines = iter(f)
    position = 0
    try:
        while True:
            POSITION_BY_ID.append(position)
            _title_with_prefix = next(lines)
            title = next(lines).split()
            for word in title:
                POSITIONAL_INDEX[base(word)].add(position)
                position += 1
            text = []
            while sentence := next(lines).split():
                for word in sentence:
                    POSITIONAL_INDEX[base(word)].add(position)
                    position += 1
                text.extend(sentence)
            WIKI_ARTICLES.append((title, text))
    except StopIteration:
        pass

In [6]:
def get_id(position):
    return bisect(POSITION_BY_ID, position) - 1

In [7]:
def find_by_phrase(base_phrase):
    positions = set.intersection(*(
        {position - offset for position in POSITIONAL_INDEX[word]}
        for offset, word in enumerate(base_phrase)
    ))
    articles_hits = {}
    for position in positions:
        id_ = get_id(position)
        title, text = WIKI_ARTICLES[id_]
        start = POSITION_BY_ID[id_]
        if id_ not in articles_hits:
            articles_hits[id_] = (title, []), (text, [])
        hit = position - start
        if hit < len(title):
            articles_hits[id_][0][1].append(hit)
        else:
            articles_hits[id_][1][1].append(hit - len(title))
    return [
        (id_, title, text)
        for id_, (title, text) in articles_hits.items()
    ]

In [8]:
def find_by_query(base_query):
    ids = set.intersection(*(
        {get_id(position) for position in POSITIONAL_INDEX[word]}
        for word in base_query
    ))
    
    queryset = set(base_query)
    results = []
    for id_ in ids:
        title, text = WIKI_ARTICLES[id_]
        results.append((
            id_,
            (title, [i for i, word in enumerate(title) if base(word) in queryset]),
            (text, [i for i, word in enumerate(text) if base(word) in queryset]),
        ))
    return results

In [9]:
def find_articles(query, phrasal):
    base_query = [base(word) for word in query]
    if not all(base_query):
        raise ValueError(f'Could not find all base forms for "{query}"')
    return (find_by_phrase if phrasal else find_by_query)(base_query)

In [69]:
TITLE_HITS_MODIFIER = 10
EXACT_MATCH_MODIFIER = 5
PHRASAL_MATCH_MODIFIER = 3
ARTICLE_ID_MODIFIER = -0.00001


def find_phrasal_matches(title, text, query):
    phrasal_matches = 0
    for i in range(len(query) - 1):
        for phrase in query[:i], query[i:]:
            window_size = len(phrase)
            if window_size <= 1:
                continue
            base_phrase = [base(word) for word in phrase]
            window = zip(*(
                [*title, *text][k:]
                for k in range(window_size)
            ))
            for words in window:
                base_words = [base(word) for word in words]
                if base_words == base_phrase:
                    phrasal_matches += 1
    return phrasal_matches


def score(result, query):
    id_, (title, title_hits), (text, _) = result
    exact_matches = len([
        qword for qword in query
        if qword.lower() in {word.lower() for word in chain(title, text)}
    ])
    phrasal_matches = find_phrasal_matches(title, text, query)
            
    return (
        len(title_hits) * TITLE_HITS_MODIFIER
        + exact_matches * EXACT_MATCH_MODIFIER
        + phrasal_matches * PHRASAL_MATCH_MODIFIER
        + id_ * ARTICLE_ID_MODIFIER
    )

def rank_results(results, query):
    scored = [(result, score(result, query)) for result in results]
    return sorted(scored, key=lambda rs: rs[1], reverse=True)

In [70]:
def highlight(text):
    return f'\033[1m\033[34m{text}\033[m'


def display(result, score, query_length):
    id_, (title, title_hits), (text, text_hits) = result
    title_copy = title[:]
    text_copy = text[:]
    for hit in title_hits:
        title_copy[hit : hit + query_length] = [
            highlight(title_copy[hit + offset]) for offset in range(query_length)
        ]
    for hit in text_hits:
        text_copy[hit : hit + query_length] = [
            highlight(text_copy[hit + offset]) for offset in range(query_length)
        ]
    return f"{highlight(score)} {' '.join(title_copy)}\n{' '.join(text_copy)}\n\n"
    

def search(query_raw, max_num_results=10):
    if query_raw.startswith('"') and query_raw.endswith('"'):
        query_raw = query_raw[1:-1]
        phrasal = True
    else:
        phrasal = False
    
    query = query_raw.split()
    results = find_articles(query, phrasal)
    ranking = rank_results(results, query)
    for (result, score), _ in zip(ranking, range(max_num_results)):
        print(display(result, score, len(query) if phrasal else 1))

In [72]:
search('kodeks karny kara za morderstwo')

16.19783 Aborcja w Chile
W Chile obowiązuje od roku 1989 całkowity zakaz aborcji ( art . 342-345 Kodeksu karnego oraz art . 119 Kodeksu służby zdrowia ) . W latach 1967-1989 była ona legalna , gdy ciąża była zagrożeniem dla życia matki . Znowelizowany Kodeks służby zdrowia wszedł w życie w 1991 roku . 21 listopada 2006 Izba Deputowanych odrzuciła projekt liberalizacji przepisów . Od 1988 roku wielokrotnie próbowano – również bez rezultatu – zwiększyć kary za nielegalną aborcję , zrównując je z tymi za morderstwo i dzieciobójstwo .


16.151510000000002 Aborcja na Malcie
Według stanu na 2003 rok maltańskie ustawodawstwo przewiduje całkowity zakaz aborcji ( art . 241-243A kodeksu karnego ) . Prawo to wprowadzono w 1981 roku - wcześniej przerywanie ciąży było legalne z powodu ścisłych wskazań medycznych . Maltańskie przepisy są najbardziej restrykcyjne w Unii Europejskiej . Za wykonanie aborcji grozi kara od osiemnastu miesięcy do trzech lat więzienia . Tej samej karze podlega kobieta cięż